# Airbyte Zendesk Support (已弃用)

注意：此连接器专用加载器已弃用。请使用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte) 代替。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于从 API、数据库和文件到数据仓库和数据湖的 ELT 管道。它拥有最大的已连接到数据仓库和数据库的 ELT 连接器目录。

此加载器将 Zendesk Support 连接器公开为文档加载器，允许您将各种对象加载为文档。

## 安装

首先，你需要安装 `airbyte-source-zendesk-support` python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-zendesk-support

## 示例

有关如何配置读取器的详细信息，请查看 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/zendesk-support/)。
配置对象应遵循的 JSON 模式可在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-zendesk-support/source_zendesk_support/spec.json](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-zendesk-support/source_zendesk_support/spec.json)。

大致结构如下：
```python
{
  "subdomain": "<your zendesk subdomain>",
  "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",
  "credentials": {
    "credentials": "api_token",
    "email": "<your email>",
    "api_token": "<your api token>"
  }
}
```

默认情况下，所有字段都存储在文档的元数据中，文本字段设置为空字符串。请通过转换读取器返回的文档来构建文档的文本。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteZendeskSupportLoader

config = {
    # your zendesk-support configuration
}

loader = AirbyteZendeskSupportLoader(
    config=config, stream_name="tickets"
)  # check the documentation linked above for a list of all streams

现在，您可以按常规方式加载文档了

In [ ]:
docs = loader.load()

由于 `load` 返回一个列表，它将阻塞直到所有文档都加载完毕。为了更好地控制这个过程，你也可以使用 `lazy_load` 方法，该方法返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请注意，默认情况下，页面内容为空，而 metadata 对象包含来自记录的所有信息。要以不同的方式创建文档，请在创建加载器时传入一个 `record_handler` 函数：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteZendeskSupportLoader(
    config=config, record_handler=handle_record, stream_name="tickets"
)
docs = loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，而不会再次加载它们。这对于数据量大且频繁更新的源非常有用。

为了利用这一点，请存储加载器的 `last_state` 属性，并在再次创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteZendeskSupportLoader(
    config=config, stream_name="tickets", state=last_state
)

new_docs = incremental_loader.load()